### Konfiguracja nowego klienta Hazelcast

In [ ]:
import hazelcast

client = hazelcast.HazelcastClient(
            cluster_name = "dev",
            cluster_members=["127.0.0.1:5701", "127.0.0.1:5702", "127.0.0.1:5703"],
        )


Debug - sprawdzenie typu klucza: 

In [ ]:
from hazelcast.aggregator import count, number_avg, max_by
from hazelcast.predicate import less_or_equal

imap = client.get_map("coinbase_trades").blocking()

entry_set = imap.entry_set()
first_key, _ = next(iter(entry_set))
print(f"Key type: {type(first_key)}") 


Key type: <class 'str'>


### Mapping SQL - prosty przykład

In [30]:
cities = client.get_map("cities").blocking()

client.sql.execute(
    """
CREATE MAPPING cities (
__key INT,
countries VARCHAR,
cities VARCHAR)
TYPE IMap
OPTIONS('keyFormat'='int', 'valueFormat'='json-flat')

    """
).result()

client.sql.execute(
    """
INSERT INTO cities VALUES
(1, 'United Kingdom','London'),
(2, 'United Kingdom','Manchester'),
(3, 'United States', 'New York'),
(4, 'United States', 'Los Angeles'),
(5, 'Turkey', 'Ankara'),
(6, 'Turkey', 'Istanbul'),
(7, 'Brazil', 'Sao Paulo'),
(8, 'Brazil', 'Rio de Janeiro')
    """
).result()

result = client.sql.execute("SELECT * FROM cities").result()


In [32]:
for row in result:
    print(row)

[__key INTEGER=2, countries VARCHAR=United Kingdom, cities VARCHAR=Manchester]
[__key INTEGER=7, countries VARCHAR=Brazil, cities VARCHAR=Sao Paulo]
[__key INTEGER=1, countries VARCHAR=United Kingdom, cities VARCHAR=London]
[__key INTEGER=3, countries VARCHAR=United States, cities VARCHAR=New York]
[__key INTEGER=6, countries VARCHAR=Turkey, cities VARCHAR=Istanbul]
[__key INTEGER=5, countries VARCHAR=Turkey, cities VARCHAR=Ankara]
[__key INTEGER=8, countries VARCHAR=Brazil, cities VARCHAR=Rio de Janeiro]
[__key INTEGER=4, countries VARCHAR=United States, cities VARCHAR=Los Angeles]


In [ ]:
# Zamknięcie klienta Hazelcast
client.shutdown()

## SQL do Ticker

In [ ]:
trades = client.get_map("coinbase_trades").blocking()

client.sql.execute(
    """

CREATE OR REPLACE MAPPING coinbase_trades
TYPE IMap
OPTIONS (
    'keyFormat' = 'varchar',
    'valueFormat' = 'json'   
);
    """
).result()


In [ ]:
result_trades = client.sql.execute(
    """
    SELECT * 
    FROM coinbase_trades 
    where cast(JSON_VALUE(this, '$.price' RETURNING varchar) as decimal) > 50000
    """).result()

for row in result_trades:
    print(row)

[__key VARCHAR=95481112030, this JSON={"type": "ticker", "sequence": 95481112030, "product_id": "BTC-USD", "price": "94272.74", "open_24h": "94281.82", "volume_24h": "2325.50982106", "low_24h": "93850", "high_24h": "95534.17", "volume_30d": "397984.00490871", "best_bid": "94272.73", "best_bid_size": "0.00859642", "best_ask": "94272.74", "best_ask_size": "0.50578886", "side": "buy", "time": "2024-12-29T15:07:37.264250Z", "trade_id": 750032370, "last_size": "0.00024691"}]
[__key VARCHAR=95481134997, this JSON={"type": "ticker", "sequence": 95481134997, "product_id": "BTC-USD", "price": "94286.59", "open_24h": "94281.82", "volume_24h": "2325.79936931", "low_24h": "93850", "high_24h": "95534.17", "volume_30d": "397984.29445696", "best_bid": "94286.58", "best_bid_size": "0.35036257", "best_ask": "94286.59", "best_ask_size": "0.00059641", "side": "buy", "time": "2024-12-29T15:08:00.021021Z", "trade_id": 750032495, "last_size": "0.029"}]
[__key VARCHAR=95481125086, this JSON={"type": "ticker"